Import required modules:

In [1]:
"""
Interactive system simulation using ipywidgets package.
Visualization functions used in this program are in the TRPL.sim.vizfuncs module.
@author: Natalia Spitha
"""
import os
import numpy as np

from KinetiKit import sim, data
from KinetiKit import artists as art
from KinetiKit import kit as kin_kit
from KinetiKit.units import units, ps, ns

from ipywidgets import interact, fixed

## Import Data
Recommended instructions: 
<br>
- Define time parameters in the time object to (see TRPL.sim.time for more information)
- Set the path directory _dir_path_ to a general folder where your files are located.
- Define the file names (including extension) and, if necessary, other subfolders that determine _file_path_.
- Define power unit and a list of powers of equal length to the list of file names. See TRPL.units for list of units.
- Ensure your files are imported correctly (check _data_from_SPCM_ in TRPL.data.lib)
<br> <br>
Optional: 
- Define directory for file containing dark counts (_dark_path_)
- Un-comment _do.dark_subtract, do.smooth,_ and _do.remove_zeros()_ if you want to apply those functions.

In [2]:
N_timepoints = 10**4
to = sim.time.linear(N=N_timepoints, period = 12.5 *ns, subsample=1)
dtime = to['array'][::to['subsample']]

#--- Creating Data Object(s)
all_data = []; keys = []
dir_path = os.getcwd()
#dir_path = r'C:\...'
dark_path = os.path.join(dir_path, 'ex_data', 'dark.asc')

filenames = [r'low_power.asc', r'mid_power.asc', r'high_power.asc',]

all_y = np.zeros((len(filenames), len(dtime))) # y data will eventually be appended to this list
power_unit = 'μW'
powers = [0.045, 0.420, 1.015] # excitation powers in defined unit

for i, filename in enumerate(filenames):
    subfolder = filename.split('_')[0] # using information from the file name to determine which folder it's in
    file_path = os.path.join(dir_path, 'ex_data', subfolder, filename)
    do = data.lib.data_from_SPCM(file_path, skip_h=354, skip_f=884, metadata=True)
    dark = data.lib.data_from_SPCM(dark_path, skip_h=354, skip_f=884, metadata=True)
    #do.dark_subtract(dark)
    do.interp(dtime)
    #do.smooth(window_length=21)
    #do.remove_zeros()
    all_y[i] = do.y

aligned_data = kin_kit.align_by_steep(all_y, dtime, value = 0.5*ns, avgnum=5)

## Define system and parameters
Instructions: 
<br>
- Select your system: _system = sim.systems.YourSystem()_
- Select whether simulations will use the _powers_ defined above (slidepower=False), or a power slider (slidepower=True)
- Define parameters as a dictionary, where the "values" are actually widget objects. You can customize the values, limits, step sizes, etc. for each slider object. <font color=red> Note: the interactive visualization function currently accepts up to 20 model parameters. </font>
<br> <br>
Optional: 
- If _slidepower_ is True, customize the power widget; see TRPL.sim.vizfuncs.powerwidget for details.

In [3]:
system = sim.systems.MonoRecX()

slidepower = False # select whether to have a slider controlling power

# Define all the parameters of the system chosen. 
# You can call the function system.params() on a cell below to quickly check which parameters are needed.

parameters = {
    
    'k_ann' : sim.vizfuncs.makeWidget('k_ann', val=1.5e9, slider='log', min_val=1e8, max_val=5e10),
                  
    'k_dis' : sim.vizfuncs.makeWidget('k_dis', val=1.4e11, slider='log', min_val=1e10, max_val=1e12),
    
    'k_rec' : sim.vizfuncs.makeWidget('k_rec', val=1e7, slider='log', min_val=1e1, max_val=1e10),
    
    # 'k_trx' : sim.vizfuncs.makeWidget('k_trx', val=0, slider='linear', min_val=0, max_val=1e8),
    
    # 'k_eth' : sim.vizfuncs.makeWidget('k_eth', val=0, slider='linear', min_val=0., max_val=1e6),
    
    # 'N_trp' : sim.vizfuncs.makeWidget('N_trp', val=0, slider='linear', min_val=0, max_val=1e6),
    
    'cs' : sim.vizfuncs.makeWidget('cs', val=0.5, slider='linear', disabled=True, readout=True)
    
        }

active_widgets = sim.vizfuncs.active_widgets(system, parameters)
ws = {f"p{i+1}": w for i, w in enumerate(active_widgets)}

powerwidget = sim.vizfuncs.powerwidget(slidepower=slidepower, unit=power_unit, power=powers[0])

[FloatLogSlider(value=1500000000.0, continuous_update=False, description='k_ann', max=10.698970004336019, min=8.0, readout_format='0.3e', step=0.01), FloatLogSlider(value=140000000000.0, continuous_update=False, description='k_dis', max=12.0, min=10.0, readout_format='0.3e', step=0.01), FloatLogSlider(value=10000000.0, continuous_update=False, description='k_rec', max=10.0, min=1.0, readout_format='0.3e', step=0.01), FloatSlider(value=0.5, continuous_update=False, description='cs', disabled=True, max=1.0, readout_format='.1f', step=0.01), <ipywidgets.widgets.interaction.fixed object at 0x0000021D62D7D748>, <ipywidgets.widgets.interaction.fixed object at 0x0000021D62D7DF08>, <ipywidgets.widgets.interaction.fixed object at 0x0000021D62D7D508>, <ipywidgets.widgets.interaction.fixed object at 0x0000021D62D7D8C8>, <ipywidgets.widgets.interaction.fixed object at 0x0000021D62D7DC88>, <ipywidgets.widgets.interaction.fixed object at 0x0000021D62D7DD08>, <ipywidgets.widgets.interaction.fixed obj

## Interactive Plot
<br>
Your interactive plot is ready (hopefully!)
<br>
Optional instructions:
- Adjust other arguments accepted by the MonoViz function, such as N_coarse. 
<br>
If the message "Array does not contain positive values!" appears, it is likely that your parameter is too fast for the chosen step size. You can increase N_coarse, as well as the N argument of the time object in the data import cell, to account for faster processes.

In [4]:
print('%s simulation:'%system.name)
GenWidget = interact(sim.vizfuncs.MonoViz,
                     system = fixed(system),
                     **ws,
                     to=fixed(to),
                     N_coarse = fixed(N_timepoints),
                     power = powerwidget,
                     irf_fwhm = fixed(50*ps),
                     data=fixed(None),  # fixed(aligned_data),
                     power_list = fixed(powers),
                     power_unit = fixed(power_unit),
                     align_by = fixed('steep'),
                     avgnum = fixed(5),
                     slidepower=fixed(slidepower),
                     xmin = fixed(0.1), xmax=fixed(None), ymin=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1], ymax=fixed(1.25)
                     )

MonoRecX simulation:


interactive(children=(FloatLogSlider(value=1500000000.0, continuous_update=False, description='k_ann', max=10.…